<a href="https://colab.research.google.com/github/Harshavalmiki/SJSU-Data-Mining-255/blob/main/255_Time_Series_with_exogenous_variable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 483.9/483.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.4/153.4 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 49.7 MB/s eta 0:0

In [2]:
#mounting google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd

Dataset: https://www.kaggle.com/datasets/himanshunakrani/cryptocurrencies-dataset
Time series Forecasting on the Cryptocurrencies and thier closing price.

In [31]:
#Link dataset
data = pd.read_csv('/content/drive/MyDrive/Pycaret_Assignment/crypto.csv', usecols=['Date', 'Close (BTC)'])

In [32]:
data.shape


(1827, 2)

In [33]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1827 entries, 0 to 1826
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         1827 non-null   object 
 1   Close (BTC)  1827 non-null   float64
dtypes: float64(1), object(1)
memory usage: 28.7+ KB


In [34]:
#Timestamp conv
data['Date']= pd.to_datetime(data['Date'])
data.set_index('Date',inplace=True)

#df.drop(['Name'],axis=1,inplace=True)
print(data.info())
print(data.head())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1827 entries, 2017-11-13 00:00:00+00:00 to 2022-11-13 00:00:00+00:00
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Close (BTC)  1827 non-null   float64
dtypes: float64(1)
memory usage: 28.5 KB
None
                           Close (BTC)
Date                                  
2017-11-13 00:00:00+00:00  6559.490234
2017-11-14 00:00:00+00:00  6635.750000
2017-11-15 00:00:00+00:00  7315.540039
2017-11-16 00:00:00+00:00  7871.689941
2017-11-17 00:00:00+00:00  7708.990234


In [35]:
# exp time
from pycaret.time_series import TSForecastingExperiment
e = TSForecastingExperiment()

In [36]:
#Monthly
data= data.asfreq('M')

In [37]:
#Setup
e.setup(data=data, target='Close (BTC)' ,fh=12, session_id=123,numeric_imputation_target="ffill", numeric_imputation_exogenous='ffill')

,Description,Value
0,session_id,123
1,Target,Close (BTC)
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(60, 1)"
5,Transformed data shape,"(60, 1)"
6,Transformed train set shape,"(48, 1)"
7,Transformed test set shape,"(12, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [38]:
#Display
e.plot_model(plot='ts')

In [39]:
#aseline models
baseline_models = e.compare_models()

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
ets,ETS,7.4901,6.6187,10997.8137,12356.6937,0.4487,0.6706,-2.8919,0.2300
naive,Naive Forecaster,7.5785,6.5413,10980.8148,11997.9439,0.4190,0.5155,-1.8850,0.1200
exp_smooth,Exponential Smoothing,7.5805,6.7111,11145.0441,12548.1376,0.4608,0.6964,-3.1437,0.1800
theta,Theta Forecaster,7.8507,6.8433,11428.2301,12639.3865,0.4364,0.5869,-2.4567,0.1600
et_cds_dt,Extra Trees w/ Cond. Deseasonalize & Detrending,8.0913,7.1743,11865.3300,13388.9870,0.4962,0.7626,-3.8253,1.0233
auto_arima,Auto ARIMA,8.4031,7.2593,12145.4304,13307.1282,0.4869,0.5819,-2.5843,0.7300
arima,ARIMA,8.4031,7.2593,12145.4304,13307.1282,0.4869,0.5819,-2.5843,0.1733
rf_cds_dt,Random Forest w/ Cond. Deseasonalize & Detrending,8.5503,7.5070,12533.2278,14010.7762,0.5666,0.8515,-5.0403,1.1500
croston,Croston,8.7008,7.4571,12585.9438,13689.1329,0.5221,0.6172,-2.8384,0.1333
knn_cds_dt,K Neighbors w/ Cond. Deseasonalize & Detrending,8.7103,7.6147,12742.8605,14166.7126,0.5017,0.7833,-3.8853,1.0667


Processing:   0%|          | 0/109 [00:00<?, ?it/s]

In [40]:
#best model
e.plot_model(baseline_models)

In [41]:
#finalize
final_model = e.finalize_model(baseline_models)

In [45]:
#creating a safe predict function as along with time we would aslo need to procide exo variables for forecasting in future
df,exp=e,data
def generate_exog_var_data_models(exp):
    exog_vars = exp.exogenous_variables
    print(f'Need the following exogenous variables for making the prediction {exog_vars}')
    #creating models to generate the future values of all exo variables
    exog_exps=[]
    exog_models=[]
    temp = data.copy()
    #drop the Close
    temp.drop(['Close'],axis=1,inplace=True)
    for exog_var in exog_vars:
      exog_exp = TSForecastingExperiment()
      exog_exp.setup(temp[exog_var],target=exog_var,fh=12,numeric_imputation_target="ffill",numeric_imputation_exogenous="ffill",session_id=152)
      best = exog_exp.compare_models()
      final_exog_model = exog_exp.finalize_model(best)
      exog_exps.append(exog_exp)
      exog_models.append(final_exog_model)
    return exog_exps,exog_models

def apply_exog_models(exp, exog_exps,exog_models):
    #use the models created to generate future predictions for exog variables
    future_exog=[
        exog_exp.predict_model(exog_model)
        for exog_exp,exog_model in zip(exog_exps,exog_models)
    ]
    future_exog = pd.concat(future_exog,axis=1)
    future_exog.columns = exp.exogenous_variables
    return future_exog



In [ ]:
#
exog_exps,exog_models = generate_exog_var_data_models(exp)

In [ ]:
future_pred_data = apply_exog_models(exp,exog_exps,exog_models)

In [ ]:
future_pred = exp.predict_model(final_model,X=future_pred_data)

In [ ]:
#save
exp.save_model(final_model,'./timeseries_model')

Transformation Pipeline and Model Successfully Saved


(ForecastingPipeline(steps=[('transformer_exogenous',
                             TransformerPipeline(steps=[('numerical_imputer',
                                                         Imputer(method='ffill',
                                                                 random_state=152))])),
                            ('forecaster',
                             TransformedTargetForecaster(steps=[('transformer_target',
                                                                 TransformerPipeline(steps=[('numerical_imputer',
                                                                                             Imputer(method='ffill',
                                                                                                     random_state=152))])),
                                                                ('model',
                                                                 ForecastingPipeline(steps=[('transformer_exogenous',
                      